In [8]:
# Sanctions × Mobilization — TWFE DiD (country & year FE; cluster by country)

import os
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# ---- 1) Load data ----
DTA_PATH = "DonganTanReplication.csv"  
df = pd.read_csv(DTA_PATH)

# Keep only rows needed for the main specs
work = df.copy()
needed = ["mm", "treat", "country", "year"]
work = work.dropna(subset=[c for c in needed if c in work.columns])

# Controls used in the .do file (only keep those actually present)
controls = ["polity", "EcGI", "lgdp", "interwar", "intrawar", "lcinc", "efindex"]
controls = [c for c in controls if c in work.columns]

# ---- 2) Define helpers ----
def fit_twfe(formula: str, data: pd.DataFrame, cluster_col: str = "country"):
    """
    Fit OLS with FE via dummies and cluster-robust SEs.
    Uses get_robustcov_results with groups aligned to model rows.
    """
    # Fit OLS
    m = smf.ols(formula=formula, data=data).fit()
    # Align clusters to the exact rows used by the model
    row_indexer = m.model.data.row_labels
    groups = data.loc[row_indexer, cluster_col]
    # Cluster-robust covariance
    m_cl = m.get_robustcov_results(cov_type="cluster", groups=groups)
    return m_cl

def pull_param(res, name="treat"):
    names = res.model.exog_names
    if name not in names:
        return np.nan, np.nan, np.nan
    idx = names.index(name)
    coef = float(res.params[idx])
    se = float(res.bse[idx])
    try:
        p = float(res.pvalues[idx])
    except Exception:
        p = np.nan
    return coef, se, p

def ci95(coef, se):
    return coef - 1.96*se, coef + 1.96*se

# ---- 3) Run models ----
# (a) TWFE, no additional controls
f0 = "mm ~ treat + C(country) + C(year)"
m0 = fit_twfe(f0, work)
b0, se0, p0 = pull_param(m0, "treat")
lo0, hi0 = ci95(b0, se0)

# (b) TWFE + controls
rhs_controls = " + ".join(controls) if controls else ""
f1 = f"mm ~ treat{(' + ' + rhs_controls) if rhs_controls else ''} + C(country) + C(year)"
# Optionally drop rows with missing controls (statsmodels will handle NaNs, but we keep N clean)
work1 = work.dropna(subset=controls) if controls else work
m1 = fit_twfe(f1, work1)
b1, se1, p1 = pull_param(m1, "treat")
lo1, hi1 = ci95(b1, se1)

# ---- 4) Summarize & save ----
rows = [
    {
        "specification": "TWFE (FE only)",
        "beta_treat": b0, "se_cluster_country": se0, "p_value": p0,
        "ci95_low": lo0, "ci95_high": hi0, "nobs": int(m0.nobs)
    },
    {
        "specification": "TWFE + controls",
        "beta_treat": b1, "se_cluster_country": se1, "p_value": p1,
        "ci95_low": lo1, "ci95_high": hi1, "nobs": int(m1.nobs)
    },
]
out = pd.DataFrame(rows)

# Save precise CSV for paper comparison
OUT_CSV = "sanctions_mobilization_results.csv"
out.to_csv(OUT_CSV, index=False)

# Pretty print
print(out.round(4))
print(f"\nSaved results to: {OUT_CSV}")

     specification  beta_treat  se_cluster_country  p_value  ci95_low  \
0   TWFE (FE only)      0.1778              0.0717   0.0144    0.0372   
1  TWFE + controls      0.2000              0.0736   0.0076    0.0557   

   ci95_high  nobs  
0     0.3183  9082  
1     0.3444  4610  

Saved results to: sanctions_mobilization_results.csv
